In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("./data/airbnb_properties.csv")

In [8]:
## Python: Find city name from latitude and longitude
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
def findCity(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        address = location.raw['address']
        city = address.get('city', '')
        if city == '':
            city = address.get('town', '')
        if city == '':
            city = address.get('village', '')
        return city
    except:
        return None
    

df["city"] = df.apply(lambda row: findCity(row['latitude'], row['longitude']), axis=1)